##### 0. 스페셜 토큰 처리?

In [1]:
# 분할 x 단어
# not_tokenized = ['#Address#', '#CarNumber#', '#CardNumber#', '#DateOfBirth#', '#Email#', 
#                 '#PassportNumber#', '#Person#', '#Person1#', '#Person2#', '#Person3#', 
#                 '#Person4#', '#Person5#', '#Person6#', '#Person7#', '#PhoneNumber#', '#SSN#']

##### 1. 발화자 / 대화 분리해서 학습

In [2]:
import pandas as pd
from koeda import EasyDataAugmentation
import re

In [6]:
# EDA 적용 함수
def augment_text_data_with_EDA(text, repetition=1):
    """입력된 문장에 대해 EDA를 통해 데이터 증강"""
    eda = EasyDataAugmentation(morpheme_analyzer="Okt")
    return eda(text, p=(0.3, 0.3, 0.3, 0.3), repetition=repetition)

# 화자와 대화 분리 함수
def split_speaker_and_text(dialogue):
    """대화 내용을 화자 태그와 분리"""
    pattern = r'(#Person\d+#): (.+)'
    lines = dialogue.split('\n')
    return [(match.group(1), match.group(2)) for line in lines if (match := re.match(pattern, line))]

# 화자와 대화 결합 함수
def join_speaker_and_text(speaker_text_pairs):
    """화자 태그와 대화 내용을 결합"""
    return '\n'.join([f'{speaker}: {text}' for speaker, text in speaker_text_pairs])

In [7]:
# 데이터 로드
df = pd.read_csv('/root/NLP/data/train.csv')
df_filtered = df[['fname', 'dialogue', 'summary', 'topic']]

# 증강 데이터를 저장할 리스트
augmented_rows = []

# EDA 적용 및 데이터 증강
for index, row in df_filtered.iterrows():
    original_text = row['dialogue']

    # 화자 태그와 대화 내용 분리
    speaker_text_pairs = split_speaker_and_text(original_text)

    # 대화 내용에 EDA 적용
    augmented_text_lists = [
        augment_text_data_with_EDA(text, repetition=3) for _, text in speaker_text_pairs
    ]

    # EDA 결과와 화자 태그 결합
    for augmented_texts in zip(*augmented_text_lists):
        new_speaker_text_pairs = [
            (speaker_text_pairs[i][0], augmented_text) 
            for i, augmented_text in enumerate(augmented_texts)
        ]
        augmented_dialogue = join_speaker_and_text(new_speaker_text_pairs)

        # 증강 데이터를 새로운 행으로 추가
        augmented_rows.append({
            'fname': row['fname'],
            'dialogue': augmented_dialogue,
            'summary': row['summary'],
            'topic': row['topic']
        })

    # 진행 상황 출력
    if index % 100 == 0:
        print(f"Processed {index + 1} rows out of {len(df_filtered)}")

# 증강 데이터를 DataFrame으로 변환
augmented_data = pd.DataFrame(augmented_rows)

# 결과 저장
augmented_data.to_csv('/root/NLP/data/aug_train.csv', index=False)

JVMNotFoundException: No JVM shared library file (libjvm.so) found. Try setting up the JAVA_HOME environment variable properly.

##### 2. df + augmented_data concat하기

##### 3. 인덱스 reset / fname 열 바꾸기(train_0 ~ train?)